<img src="https://creative.lbl.gov/wp-content/uploads/sites/3/2020/07/6_BL_Horiz_Rev_rgb.png"
     width="500px"
     alt="LBL logo"
     style="vertical-align:middle"/>


# Cloud microphysics training and aerosol inference with the Fiats deep learning library

**Authors:** [Damian Rouson](mailto:rouson@lbl.gov), [Zhe Bai](mailto:zhebai@lbl.gov), [Dan Bonachea](mailto:dobonachea@lbl.gov),  [Katherine Rasmussen](mailto:krasmussen@lbl.gov)

**Keywords:** deep learning, Fortran, cloud microphysics, aerosols, surrogate model, neural network

----------

## Abstract
This notebook presents two atmospheric sciences demonstration applications in the [Fiats](https://go.lbl.gov/fiats) deep learning software repository. The first, `train-cloud-microphysics`, trains a neural-network cloud microphysics surrogate model that has been integrated into the [Berkeley Lab fork](https://go.lbl.gov/icar) of the Intermediate Complexity Atmospheric Research (ICAR) model. The second, `infer-aerosol`, performs parallel inference with an aerosol dynamics surrogate pretrained in PyTorch using data from the Energy Exascale Earth System Model ([E3SM](https://e3sm.org)). This notebook presents the program statements involved in using Fiats for aerosol inference and microphysics training. In order to also give the interested reader direct experience with using Fiats for these purposes, the notebook details how to run two simpler example programs that serve as representative proxies for the demonstration applications.  Both proxies are also example programs in the Fiats repository. The microphysics training proxy is a self-contained example requiring no input files.  The aerosol inference proxy uses a pretrained aerosol model stored in the Fiats JavaScript Object Notation (JSON) file format and hyperlinked into this notebook for downloading, importing, and using to perform batch inference calculations with Fiats.

Introduction
------------
### Background
Fortran programs serve an important role in earth systems modeling from weather and climate prediction {cite}`skamarock2008description,mozdzynski2015partitioned` to wildland fire modeling {cite}`vanella2021multi` and terrestrial ecosystem simulation {cite}`shi2024functionally`.   The cost of performing ensembles of runs of such complex, multiphysics applications at scale inspires investigations into replacing model components with neural-network surrogates in such earth systems as groundwater {cite}`asher2015review` and oceans {cite}`partee2022using`.  For the surrogates to be useful, they must provide comparable or better accuracy and execution speed as the physics-based components that the surrogates replace.  The prospects for achieving faster execution depend on several properties of the surrogates, including their size, complexity, architecture, and implementation details.  Implementing the surrogates in the language of the supported application reduces interface complexity and increases runtime efficiency by reducing the need for wrapper procedures and data structure transformations.
Inspired partly by these concerns, native Fortran software packages are emerging to support the training and deployment of neural-network surrogate models.  Two examples are neural-fortran {cite}`curcic2019parallel` and ATHENA {cite}`taylor2024athena`.  This jupyter notebook presents two demonstration applications in the Fiats deep learning library {cite}`rouson2025automatically`: one trains a cloud microphysics surrogate for the Intermediate Complexity Atmospheric Research (ICAR) model {cite}`gutmann2016intermediate` and the other uses a pretrained surrogate for aerosols in the Energy Exascale Earth Systems Model (E<sup>3</sup>SM) {cite}`golaz2019doe`.

Fiats, an acronym that expands to "Functional inference and training for surrogates" or "Fortran inference and training for science," targets high-performance computing applications (HPC) in Fortran 2023. Fiats provides novel support for functional programming styles by providing inference and training procedures declared to be `pure`, a language requirement for invoking a procedure inside Fortran's loop-parallel construct: `do concurrent.` Because pure procedures clarify data dependencies, at least four compilers are currently capable of automatically parallelizing `do concurrent` on central processing units (CPUs) or graphics processing units (GPUs): the Intel Fortran Compiler `ifx`, LLVM `flang`, the NVIDIA HPC Fortran compiler `nvfortran`, and the HPE Cray Compiling Environment Fortran compiler `crayftn`.  Fiats supports `flang` and partially supports `ifx` with only one known test failure when building with `ifx` at the time of this writing.  At the time of this writing, Fiats supports `flang` and offers automatic parallelization of batch inference calculations with strong scaling trends comparable to those achievable with OpenMP directives {cite}`rouson2025automatically`. Work is under way to support `ifx`.

Fiats provides a derived type that encapsulates neural-network parameters and provides generic bindings for invoking inference functions and training subroutines of various precisions. A novel feature of the Fiats design is that all procedures involved in inference and training have the `non-overridable` attribute, which eliminates the need for dynamic dispatch at call sites. In addition to simplifying the structure of the resulting executable program and potentially improving performance, we expect this feature to enable the automatic offload of inference and training to GPUs.

### Objectives
The primary objectives of this notebook are to describe the use of Fiats in the `infer-aerosol` and `train-cloud-microphysics` demonstration applications and to provide the user experience with Fiats by detailing how to run simpler examples programs that use Fiats in similar ways to the demonstration applications and thus serve as reasonable proxies for the demonstration applications.  The aerosol inference proxy performs batch inference calculations using a pretrained aerosol model {cite}`bai2024deep` provided via hyperlink in this notebook. The cloud microphysics proxy trains a deep neural network to model an ICAR SB04 cloud microphysics model function that predicts the saturated mixing ratio from a given pressure and temperature.  The [Methodology](#methodology) section of this notebook describes the use of Fiats in the two demonstration applications.  The [Discussion](#discussion) section explains how to run the two proxies.   We expect that the reader familiar with recent Fortran standards will learn the program statements required to use Fiats.  We further expect that an interested reader who installs the prerequisite build system and compiler will learn to run programs locally using Fiats.

## Methods
### Getting started
With the `tree` utility installed, the following `bash` shell commands will download the Fiats repository, checkout the `git` commit used in writing this notebook, and show the Fiats directory tree:
```bash
git clone --branch sea-iss-2025 git@github.com:berkeleylab/fiats 
cd fiats
tree -d
.
├── demo
│   ├── app
│   ├── include -> ../include
│   ├── src
│   └── test
├── doc
│   └── uml
├── example
│   └── supporting-modules
├── include
├── scripts
├── src
│   └── fiats
└── test
```
where the `git clone` command checks out the `sea-iss-2025` commit used for developing this notebook.

The `src` directory contains the source comprising the Fiats library that programs link against to access Fiats functionality by invoking the library's procedures.  As such, `src` contains the only files that a Fiats user needs and it contains no main programs.  The `fiats_m` `module`in `src/fiats_m.f90` contains all user-facing Fiats functions, procedures, derived types, and constants.  The `src/fiats` subdirectory contains the definitions of all of these entities in addition to private internal implementation components.  For a program to access Fiats entities, it would suffice for a `use fiats_m` statement to appear in any program unit or subprogram that requires Fiats.

Apart from the library, the Fiats `git` repository contains main programs that demonstrate how to use Fiats.  The main programs are in two subdirectories: 
1. `example/` contains relatively short and mostly self-contained programs and
2. `demo/app/` contains demonstration applications developed with collaborators for production use.
The next three subsections describe programs in these locations.

### Demonstration application: aerosol inference
The `demo/app/infer-aerosol.f90` program demonstrates the use of Fiats to predict aerosol variables for E<sup>3</sup>SM.   The following statement provides access to all Fiats entities employed by the program:
```fortran
   use fiats_m, only : unmapped_network_t, tensor_t, double_precision, double_precision_file_t
```
where the `unmapped_network_t` derived type encapsulates a neural network that performs no mappings on input and output tensors, `tensor_t` encapsulates network input and output tensors, `double_precision` is a kind type parameter that specifies the desired precision, and the `double_precision_file_t`  derived type provides a file abstraction that determines how numerical values in model files will be interpreted.  Because Fiats focuses on surrogate models that must be compact in order to be competitive with the physics-based models they replace, Fiats uses a JSON file format for its human readability because we have found the ability to inspect network parameters visually helpful in the early stages of experimenting with new algorithms.  Users with models trained in PyTorch can use the Fiats companion network export software [Nexport](https://github.com/berkeleylab/nexport) to export models to the Fiats JSON format.

After chores such as printing usage information if a user omits a required command-line argument, the following object declaration demonstrates the first direct use of Fiats in the program: 
```fortran
   type(unmapped_network_t(double_precision)) neural_network
```
Fiats uses derived type parameters -- specifically kind type parameters -- so that one neural-network type can be used to declare objects with any supported `kind` parameter.  Currently, the supported `kind` parameters are `default_real` and `double_precision`, corresponding to the chosen compiler's `real` (with no specified `kind` parameter) and `double precision` types.  Fiats types with a kind type parameter provide a default initialization of the parameter to `default_real`.  

A later line defines the object:
```fortran
   neural_network = unmapped_network_t(double_precision_file_t(path // network_file_name))
```
where `unmapped_network_t` appears in this context as a generic interface patterned after Fortran's structure constructors that define new objects. Because the JSON specification does not differentiate types of numbers (e.g., JSON does not distinguish integers from real numbers), using the Fiats `double_precision_file_t` type specifies how to interpret values read from the model file.

Similarly, the later line 
```fortran
   type(tensor_t(double_precision)), allocatable :: inputs(:), outputs(:)
```
specifies the precision used for tensor objects and the `tensor_t` generic interface in the following statement
```fortran
   inputs(i) = tensor_t(input_components(i,:))
```
resolves to an invocation of a function that produces a double-precision object because of a declaration earlier in the code (not shown here) that declares `input_components` to be of type `double precision`.  Ultimately, inference happens by invoking a type-bound `infer` procedure on the `neural_network` object and providing `tensor_t` input object to produce the corresponding outputs:
```fortran
   !$omp parallel do shared(inputs,outputs,icc)
   do i = 1,icc
     outputs(i) = neural_network%infer(inputs(i))
   end do
   !$omp end parallel do
```
where we parallelize the loop using OpenMP directives.  Alternatively, the Fiats `example/concurrent-inferences.F90` program invokes `infer` inside a `do concurrent` construct, taking advantage of `infer` being `pure`.  This approach has the advantage that compilers can automatic parallelize the iterations without OpenMP directives.  Besides simplifying the code, switching to `do concurrent` means the exact same source code can run in parallel on a CPU or a GPU without change.  With most compilers, switching from running on one device to another requires simply recompiling with different flags.  See {cite:t}`rouson2025automatically`  for more details on automatically parallelizing inference, including strong scaling results on one node of the Perlmutter supercomputer at the National Energy Research Scientific Computing (NERSC) Center.

The remainder of `infer-aerosol` contains problem-specific statements not directly related to the use of Fiats and is therefore beyond the scope of this notebook.

### Demonstration application: microphysics training
Training a neural network is an inherently more involved process than using a neural network for inference.  As such, `train-cloud-microphysics` uses a larger number of of Fiats entities:
```fortran
use fiats_m, only : tensor_t, trainable_network_t, input_output_pair_t, mini_batch_t, &
    tensor_map_t, training_configuration_t, training_data_files_t, shuffle
```
where only the `tensor_t` type intersects with the set of entities that `infer-aerosols` uses.  The remaining entities in the above `use` statement all relate to training neural networks.

The `trainable_network_t` type extends the `neural_network_t` type and thus offers the same type-bound procedures by inheritance. 
Outwardly, `trainable_network_t` differs from `neural_network_t` only in that the former provides public `train` and `map_to_training_ranges` generic bindings that the latter lacks.  Calling `train` performs a forward pass followed by a back-propagation pass that adjusts the neural-network weights and biases.  If the network input and output ranges for training differ from the corresponding tensor values for the application (e.g., we often find it useful to map tensor values to the unit interval [0,1] for training), then calling `map_to_training_ranges` performs the desired transformation and the resulting `tensor_map_t` type encapsulates the forward and inverse mappings.  Privately, the `trainable_network_t` type stores a `workspace_t` object containing a training scratchpad that gets dynamically sized in a way that is invisible to Fiats users.  Hiding this implementation detail without necessitating the definition of `neural_network_t` components needed only for training is the primary reason `trainable_network_t` exists.

The `input_output_pair_t` derived type encapsulates training-data pairings ensuring a one-to-one connection between `tensor_t` inputs and outputs as required for supervised learning {cite}`goodfellow2016deep`.  The `mini_batch_t` type supports the formation of `input_output_pair_t` subgroups. The ability to form mini-batches and the listed `shuffle` procedure combine to facilitate the implementation of the foundational stochastic gradient descent optimization algorithm for training.

Finally, the `training_configuration_t` and `training_data_files_t` types encapsulate file formats that Fiats users employ to define training hyperparameters (e.g., learning rate) and to specify the series of file names that contain training data.  With all of the aforementioned derived types in place, `train-cloud-microphysics` uses a capability of the external Julienne framework {cite}`julienne` to group the training data into bins:

```fortran
bins = [(bin_t(num_items=num_pairs, num_bins=n_bins, bin_number=b), b = 1, n_bins)]
```
and then these bins are shuffled into new mini-batch subsets in each pass (epoch) through the data set:

```fortran
do epoch = first_epoch, last_epoch

  if (size(bins)>1) call shuffle(input_output_pairs) ! set up for stochastic gradient descent
    mini_batches = [(mini_batch_t(input_output_pairs(bins(b)%first():bins(b)%last())), b = 1, size(bins))]

    call trainable_network%train(mini_batches, cost, adam, learning_rate)
```
which completes the presentation of essential Fiats capabilities employed by `train-cloud-microphysics`.

## Discussion
This section aims to provide the interested reader with experience in running programs that use Fiats for predicting atmospheric aerosol dynamics and for training a cloud microphysics model. 

### Build system and compiler installation
Building Fiats requires the Fortran Package Manager ([`fpm`](https://github.com/fortran-lang/fpm)) and the LLVM [`flang`](https://github.com/llvm/llvm-project) Fortran compiler.  The commands in this notebook were tested with `fpm` 0.11.0 and `flang` 20.1.2.  It might be easiest to install `fpm` and `flang` using package managers -- for example, using the [Homebrew]() command `brew install fpm flang` on macOS or the command `sudo apt install fortran-fpm flang-20` on Ubuntu Linux.  Alternatively, if you have the GNU Compiler Collection ([GCC](https://gcc.gnu.org)), an especially easy way to install `fpm` is to compile the single file that is included on each `fpm` release page and that contains all of the source code for `fpm` (which is written in Fortran) concatenated into one file.  On the `fpm` 0.11.0 release page, for example, that file is [fpm-0.11.0.F90](https://github.com/fortran-lang/fpm/releases/download/v0.11.0/fpm-0.11.0.F90).  Downloading that file to an otherwise empty directory (because compiling it produces many files, all but one of which can be deleted after compiling), compiling with a command such as `gfortran -o fpm fpm-0.11.0.F90`, and moving the resulting `fpm` executable file to a directory in your `PATH` gives you a working installation of `fpm`.

With `fpm` and `flang` installed, and the repository cloned as explained in section [Getting started](#getting-started) section of this notebook, and with your present working directory set to anywhere in the clone, you can build and test Fiats with the command:
```bash
fpm test --compiler flang-new --flag -O3
```
where we invoke `flang` via the alias `flang-new` for historical reasons.  We expect that an upcoming version of `fpm` will recognize `flang` as LLVM `flang`.  If everything succeeds, the trailing output from the latter command should be:
```
_________ In total, 37 of 37 tests pass. _________
```
at which point it will be possible to run any of the programs in the Fiats example programs.

### Running microphysics training
As explained in the [Objectives](#objectives) section of this notebook, this section uses the `learn-saturated-mixing-ratio` program in the `example/` subdirectory as a proxy for the much more involved `train-cloud-microphysics` demonstration application in the `demo/app` subdirectory.  The example program trains a neural network surrogate for the function defined in `example/supporting-modules/saturated-mixing-ratio.f90`.  The function result is the saturated mixing ratio, a thermodynamic variable corresponding to the maximum amount of water vapor that the air at a given location can hold in the gaseous state without condensing to liquid.   The function has two arguments: temperature and pressure. The function was extracted from ICAR's SB04 simple microphysics model, refactored to accept arguments mapped to the unit interval [0,1], and then wrapped by a function that accepts and returns `tensor_t` objects from which the Fiats `infer` function extracts model inputs and the corresponding outputs.  The resulting trained neural network stores the physics-based (SB05) model input and output extrema for purposes of capturing the mapping function that transforms data between the application data range and the model data range ([0,1]). 

The `learn-saturated-mixing-ratio` example program uses Fiats entities that form a subset of those used by `train-cloud-microphysics`:
```fortran
   use fiats_m, only : trainable_network_t, mini_batch_t, tensor_t, input_output_pair_t, shuffle
```
which demonstrates why the example is a simpler proxy for the demonstration application. Because the [Methods](#methods) section of this notebook details the demonstration application's use of Fiats, the current section skips over example program details and instead focuses on running the example.

All of the programs in the Fiats `example/` and `demo/app/` subdirectories are designed to print helpful usage information when run without all required command-line arguments.  Hence, running the provided programs without arguments is one way to find out the required arguments: 
```bash
   fpm run \
     --example learn-saturated-mixing-ratio \
     --compiler flang-new \
     --flag -O3
```
which generates the trailing output
```bash
Fortran ERROR STOP: 

Usage:
  fpm run \
  --example learn-saturated-mixing-ratio \
  --compiler flang-new \
  --flag "-O3" \
  -- --output-file "<file-name>"

<ERROR> Execution for object " learn-saturated-mixing-ratio " returned exit code  1
<ERROR> *cmd_run*:stopping due to failed executions
STOP 1
```
where `fpm` commands contain command-line arguments for the running program after double-dashes (`--`) so the above message indicates that the program requires command-line arguments of the form `--output-file "<file-name>"`.

The following command runs the program again with the required argument:
```bash
   fpm run \
     --compiler flang-new \
     --flag -O3 \
     --example learn-saturated-mixing-ratio \
     -- --output-file saturated-mixing-ratio.json 
```
which should produce leading output like the following:
```
 Initializing a new network
         Epoch | Cost Function| System_Clock | Nodes per Layer
         1000    0.77687E-04     6.0089      2,4,72,2,1
         2000    0.60092E-04     12.062      2,4,72,2,1
         3000    0.45148E-04     18.110      2,4,72,2,1
         4000    0.33944E-04     24.253      2,4,72,2,1
```
showing the cost function decreasing with increasing numbers of epochs for a neural network that accepts `2` inputs (normalized pressure and temperature) and uses three hidden layers of width `4`, `72`, and `2` to produce `1` output (saturated mixing ratio).

The program uses a cost-function tolerance of of `1.0E-08`, which takes a very long time to attain when compiled with LLVM `flang` 20.  (An earlier version of Fiats, 0.14.0, supports compiling with `gfortran`, which produces executable programs that run approximately 20x faster.  We anticipate supporting `gfortran` again in a future release after bugs in `gfortran`'s support for kind type parameters have been fixed. We also anticipate that future releases of LLVM `flang`, one of the newest Fortran compilers, will improve in the ability to optimize code and generate faster programs.)

To gracefully shut down the example program, issue the command `touch stop`, which creates an empty file named `stop`.  The program periodically checks for this file, halts execution if the file exists, and prints a table of model inputs along with the actual and desired model outputs.  [Fig. 1](sat-mix-rat) compares the surrogate and physics-based model output surfaces over the unit square domain $[0,1]^2$. It demonstrates that the two surfaces are visually indistinguishable, except that whichever of the two surface colors shows at a given point indicates which surface would be visible from the given viewing angle.  Viewed from above, for example, the color corresponding to whichever surface is slightly higher than the other shows.  From below, whichever is slightly lower shows.

```{figure} ../assets/saturated-mixing-ratio-surface-plot2.png
:name: sat-mix-rat
:align: center

Surface plots of the actual (green) and desired (blue) saturated mixing ratio model outputs.
```

### Running aerosol inference
This section of the notebook uses the `concurrent-inferences` program in the `example/` subdirectory as a proxy for the more involved `infer-aerosol` demonstration application in the `demo/app` subdirectory.  This example program performs batches of inferences taking a three-dimensional (3D) array of `tensor_t` input objects and producing a 3D array of `tensor_t` output objects.  The sizes of the 3D arrays is representative of the grids used in an ICAR production run. 

Run the proxy with no command-line arguments for the program itself:
```bash
   fpm run --example concurrent-inferences --compiler flang-new --flag -O3
```
which should yield the usage information:
```bash
Usage:
  fpm run \
    --example concurrent-inferences \
    --compiler flang-new --flag -O3 \
    -- --network "<file-name>" \
    [--do-concurrent] [--openmp] [--elemental] [--double-precision] [--trials <integer>]
where <> indicates user input and [] indicates an optional argument.
```
where the first three optional arguments specify a strategy for iterating across the batch: Fortran's loop-parallel `do concurrent` construct, OpenMP multithreading, or an `elemental` procedure that operates on whole `tensor_t` arrays or array slices in one `infer` invocation.  If none of the first three optional arguments exists on the command line, then all three execute.  The fourth optional argument decides whether to additionally perform inference using double precision. The final optional argument determines the number of times each strategy will execute.  See {cite}`rouson2025automatically` for a discussion of the performance of each of these approaches.

Before running `concurrent-inferences`, download the pretrained aerosol model file [model.json](./assets/model.json) and save it to the root directory of your Fiats clone. 
To import the pretrained model and run the program, enter the following command:
```bash
  fpm run \
    --example concurrent-inferences \
    --compiler flang-new --flag -O3 \
    -- --network model.json
```
which should yield trailing output of the form:
```
 Constructing a new neural_network_t object from the file model.json
 Defining an array of tensor_t input objects with random normalized components
 Performing 1250565  inferences inside `do concurrent`.
 Elapsed system clock during `do concurrent` inference:  42.464405
 Performing 1250565  inferences inside `omp parallel do`.
 Elapsed system clock during `OpenMP` inference:  43.313417
 Performing elemental inferences inside `omp workshare`
 Elapsed system clock during `elemental` inference:  41.631988
 Constructing a new neural_network_t object from the file model.json
 Defining an array of tensor_t input objects with random normalized components
 Performing double-precision inference inside `do concurrent`
 Elapsed system clock during double precision concurrent inference:  66.173744
 variable          mean           stdev
 t_dc     42.464405 0.
 t_omp    43.313417 0.
 t_elem   41.631988 0.
 t_dp_dc  66.173744 0.
```
In interpreting these timings, note that Homebrew installs LLVM `flang` without OpenMP support and `flang`'s capability for automatically parallelizing of `do concurrent` did not make it into `flang` version 20, but should appear in `flang` version 21.  Lastly, compilers do not yet parallelize array statements inside OpenMP's `!$omp workshare` blocks.  {cite}`rouson2025automatically` present results with OpenMP support and with automatic parallelization of `do concurrent` enabled.

[Fig. 2](aerosol-viz) visualizes predictions of the accumulation-mode aerosol concentration made by the aerosol model that we used with the `infer-aerosol` demonstration application.  The visualization is produced by software unrelated to Fiats and is provided for purposes of understanding the model data the model produces.
```{figure} ../assets/ncvis_output_0104_a1.png
:name: aerosol-viz
:align: center

A global view of the concentration of accumulation-mode aerosol particles as predicted by the model used in this section of the notebook.
```



## Conclusions
This notebook introduces the reader to program statements that demonstration applications can use to access Fiats functionality for training a cloud microphysics surrogate model, or to perform batch inference using an aerosol surrogate.  After providing information on installing the requisite compiler (LLVM `flang`) and build system (`fpm`), the notebook  walks the reader through the process of running simplified example proxy programs modeled after the demonstration applications. Related visualizations demonstrate  the accuracy of the microphysics model trained by Fiats and the richness of the structures representable with the pretrained aerosol model.

## References
```{bibliography}
```